In [1]:
import os
import re
import datetime
import pandas as pd
import numpy as np
from selenium import webdriver
from tika import parser
from bs4 import BeautifulSoup
import requests

In [2]:
last_page = 23
pages = np.arange(1,last_page +1)

path = "../data/"

if not os.path.exists(path):
    os.makedirs(path)

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
prefs = {"profile.default_content_settings.popups": 0,
             "download.default_directory": 
                        path,
             "directory_upgrade": True}
options.add_experimental_option("prefs", prefs)
driver=webdriver.Chrome("./chromedriver", options=options)

for page in pages:
    url = 'http://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(page)
    driver.get(url)
    
    li_tags = driver.find_elements_by_css_selector('div.type2 li a.secretSet')
    n_li_tags = len(li_tags)
    for idx in range(n_li_tags):
        li_tag = li_tags[idx]
        li_tag.click()
        name = driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li:nth-child(1) > a:nth-child(1)').text
        if name.split('.')[1] == 'hwp':
            try:
                driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li:nth-child(2) > a:nth-child(1)').click()
            except:
                driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li > a:nth-child(1)').click()
        else:
            driver.find_element_by_css_selector('#board > div > dl > dd > div > ul > li:nth-child(1) > a:nth-child(1)').click()
        driver.back()
        li_tags = driver.find_elements_by_css_selector('div.type2 li a.secretSet') # 페이지가 새로 로드되면 li_tags를 잃어버리므로 다시 할당


In [4]:
path = "../data/"
file_list = os.listdir(path)

file_list_pdf = [file for file in file_list if file.endswith(".pdf")]
file_list_hwp = [file for file in file_list if file.endswith(".hwp")]

file_list = file_list_pdf + file_list_hwp
print(file_list[:10], len(file_list), sep='\n')

['금융통화위원회 의사록(2022년도 제14차).pdf', '금융통화위원회 의사록(2022년도 제18차).pdf', '제13차 금통위 의사록 (1).pdf', '제24차 금통위 의사록 (3).pdf', '제23차 금통위 의사록 (2).pdf', '제9차 금통위 의사록 (6).pdf', '제22차_금통위 의사록 (1).pdf', '제16차 금통위 의사록 (2).pdf', '제12차_금통위 의사록 (2).pdf', '제11차 금통위 의사록 (3).pdf']
260


In [5]:
dates = []
contents = []
titles = []

i = 1
for idx, file in enumerate(file_list):
    
    raw = parser.from_file('../data/{}'.format(file))
    date2 = re.search('\d{4}[가-힣\s]+\d+월? \d+일?', raw['content']).group()
    date2 = '.'.join(re.findall('[0-9]+',date2))
        
    title = file.split('.')[0]

    dates.append(date2)
    titles.append(title)
    contents.append(raw['content'])

    i += 1
    if i % 10 == 0:
        print(i,'/',len(file_list), '완료')
print('모든 작업 완료')

10 / 260 완료
20 / 260 완료
30 / 260 완료
40 / 260 완료
50 / 260 완료
60 / 260 완료
70 / 260 완료
80 / 260 완료
90 / 260 완료
100 / 260 완료
110 / 260 완료
120 / 260 완료
130 / 260 완료
140 / 260 완료
150 / 260 완료
160 / 260 완료
170 / 260 완료
180 / 260 완료
190 / 260 완료
200 / 260 완료
210 / 260 완료
220 / 260 완료
230 / 260 완료
240 / 260 완료
250 / 260 완료
260 / 260 완료
모든 작업 완료


In [6]:
df = pd.DataFrame({'date':dates,'title':titles,'content':contents})

df.date = pd.to_datetime(df.date)
df = df.sort_values(by='date',ascending=True).reset_index(drop=True)

df

,date,title,content
0,2009-11-12,제24차 금통위 의사록 (6),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,2009-11-26,제25차 금통위 의사록 (6),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,2009-12-10,제26차 금통위 의사록 (2),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,2009-12-24,제27차 금통위 의사록 (1),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,2010-01-07,제1차 금통위 의사록 (8),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
...,...,...,...
255,2022-08-25,금융통화위원회 의사록(2022년도 제16차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
256,2022-09-08,금융통화위원회 의사록(2022년도 제17차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
257,2022-09-22,금융통화위원회 의사록(2022년도 제18차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
258,2022-10-12,금융통화위원회 의사록(2022년도 제19차),\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


In [7]:
df.to_csv('../data/MPB.csv', encoding='utf-8-sig', header=True, index=False)

In [8]:
df.title[:20]

0     제24차 금통위 의사록 (6)
1     제25차 금통위 의사록 (6)
2     제26차 금통위 의사록 (2)
3     제27차 금통위 의사록 (1)
4      제1차 금통위 의사록 (8)
5      제2차 금통위 의사록 (5)
6      제3차 금통위 의사록 (7)
7      제4차 금통위 의사록 (3)
8      제6차 금통위 의사록 (7)
9      제7차 금통위 의사록 (7)
10     제8차 금통위 의사록 (5)
11     제9차 금통위 의사록 (6)
12    제10차 금통위 의사록 (2)
13    제12차 금통위 의사록 (5)
14    제13차 금통위 의사록 (7)
15    제14차 금통위 의사록 (5)
16    제15차 금통위 의사록 (6)
17    제16차 금통위 의사록 (4)
18    제17차 금통위 의사록 (5)
19    제18차 금통위 의사록 (7)
Name: title, dtype: object